In [ ]:
load("/uufs/chpc.utah.edu/common/HIPAA/u1264408/u1264408/Git/SEMIColon/scripts/pks/Nissle_processed.RData")
library(tidyverse)
library(reshape2)
library(magrittr)
library(lemon)
library(BSgenome)
library(MutationalPatterns)
library(plyr)
library(data.table)
library(vroom)
library(ggpubr)
library(cowplot)
library(patchwork)
library(BSgenome.Hsapiens.UCSC.hg38)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(sjPlot)
library(org.Hs.eg.db)
library(GenomicFeatures)
library(AnnotationDbi)
library(org.Hs.eg.db)

ref_genome = "BSgenome.Hsapiens.UCSC.hg38"
genes = genes(TxDb.Hsapiens.UCSC.hg38.knownGene)
load("vcfs.RData")

- Coding (snv, indel fractions, signature refit)
- Driver genes (snv, indel fractions, signature refit)

GET CODING SNVs (also have gene functionality but yeah let's worry about that later)

In [ ]:
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

cds <- cdsBy(txdb, by = "gene")

In [ ]:
### Genes aren't specific enough here so we will use coding sequences
annotate_with_genes <- function(df, genes) {

  gr <- GRanges(
    seqnames = df$chr,
    ranges = IRanges(start = df$position, end = df$position),
    strand = df$strand
  )

  hits <- findOverlaps(gr, genes)

  gene_ids <- mcols(genes)$gene_id[subjectHits(hits)]

  symbols <- mapIds(
    org.Hs.eg.db,
    keys = gene_ids,
    column = "SYMBOL",
    keytype = "ENTREZID",
    multiVals = "first"
  )

  # initialize column
  df$gene_symbol <- NA_character_

  # assign symbols
  df$gene_symbol[queryHits(hits)] <- symbols

  df
}

In [ ]:
annotate_with_coding_genes <- function(df, txdb) {

  gr <- GRanges(
    seqnames = df$chr,
    ranges   = IRanges(start = df$position, end = df$position),
    strand   = df$strand
  )

  cds <- cdsBy(txdb, by = "gene")

  hits <- findOverlaps(gr, cds, ignore.strand = TRUE)

  # initialize column
  df$gene_symbol_coding <- NA_character_

  # 🚫 no coding hits → return early
  if (length(hits) == 0) {
    return(df)
  }

  gene_ids <- names(cds)[subjectHits(hits)]

  symbols <- mapIds(
    org.Hs.eg.db,
    keys      = gene_ids,
    column    = "SYMBOL",
    keytype   = "ENTREZID",
    multiVals = "first"
  )

  df$gene_symbol_coding[queryHits(hits)] <- symbols

  df
}

In [ ]:
contexts_test <- contexts[c(1, 2)]

In [ ]:
contexts_test_coding <- lapply(contexts_test, annotate_with_coding_genes, txdb = txdb)

In [ ]:
contexts_coding <- lapply(contexts, annotate_with_coding_genes, txdb = txdb)

In [ ]:
save.image(file = "colibactincoding.RData")

In [ ]:
snv_coding <- data.frame(
  sample = names(contexts_coding),
  n_with_gene = sapply(
    contexts_coding,
    function(df) sum(!is.na(df$gene_symbol_coding))
  ),
  row.names = NULL
)

In [ ]:
contexts_coding$'029_DC_T'

In [ ]:
sum(snv_coding$n_with_gene)

In [ ]:
snv_coding

Now that I have CODING SNVs, let's fitler for the PKS

In [ ]:
contexts_coding_filtered <- lapply(
  contexts_coding,
  function(df) df[!is.na(df$gene_symbol_coding), ]
)

In [ ]:
context_list = list(normal = contexts_coding_filtered[categories$injection == "normal"],
                    adenoma = contexts_coding_filtered[categories$injection == "adenoma"],
                    carcinoma = contexts_coding_filtered[categories$injection == "carcinoma"])

In [ ]:
contexts_TN_sample <- context_list %>%
  purrr::flatten() %>%
  purrr::map(~ .x %>%
    distinct() %>%
    filter(grepl("^T", type))
  )

contexts_TN_sample <- setNames(
  contexts_TN_sample,
  names(contexts_TN_sample)
)

In [ ]:
ext_context_sample <- rbindlist(
  contexts_TN_sample,
  idcol = "name"
)

In [ ]:
TRIPLETS_48 = TRIPLETS_96[49:96]
SBS88_TN = as.data.table(signatures) %>% dplyr::slice(49:96) %>% pull("SBS88")

In [ ]:
ext_context_sample <- ext_context_sample %>%
  mutate(
    pos34 = substr(context, 7, 8),
    trinucleotide = factor(trinucleotide, levels = TRIPLETS_48),
    select = factor(
      ifelse(pos34 == "AA", "AA", "other"),
      levels = c("other", "AA")
    ),
  )

In [ ]:
snv_load <- ext_context_sample[
  select == "AA",
  .(n_AA = .N),
  by = name
]

In [ ]:
snv_load

In [ ]:
coding_snv_burden <- merge(snv_load, snv_coding, by.x = "name", by.y = "sample")

In [ ]:
write.csv(coding_snv_burden, "colibactin_snv_burden_coding.csv", row.names = FALSE)

Let's get coding indels + fraction

In [ ]:
vcfs_indel

In [ ]:
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

cds_gr <- unlist(
  cdsBy(txdb, by = "gene"),
  use.names = TRUE   # keep ENTREZ IDs as names
)

In [ ]:
annotate_indels_coding <- function(gr, cds_gr) {

  # initialize metadata
  mcols(gr)$coding      <- FALSE
  mcols(gr)$gene_id     <- NA_character_
  mcols(gr)$gene_symbol <- NA_character_

  hits <- findOverlaps(gr, cds_gr, ignore.strand = TRUE)

  if (length(hits) == 0)
    return(gr)

  gene_ids <- names(cds_gr)[subjectHits(hits)]

  symbols <- mapIds(
    org.Hs.eg.db,
    keys      = gene_ids,
    column    = "SYMBOL",
    keytype   = "ENTREZID",
    multiVals = "first"
  )

  # mark coding
  mcols(gr)$coding[queryHits(hits)] <- TRUE

  # assign gene info
  mcols(gr)$gene_id[queryHits(hits)]     <- gene_ids
  mcols(gr)$gene_symbol[queryHits(hits)] <- symbols

  gr
}

In [ ]:
annotate_indels_genic <- function(gr, genes) {

  # initialize metadata
  mcols(gr)$genic       <- FALSE
  mcols(gr)$gene_id     <- NA_character_
  mcols(gr)$gene_symbol <- NA_character_

  hits <- findOverlaps(gr, genes, ignore.strand = TRUE)

  if (length(hits) == 0)
    return(gr)

  gene_ids <- mcols(genes)$gene_id[subjectHits(hits)]

  symbols <- mapIds(
    org.Hs.eg.db,
    keys      = gene_ids,
    column    = "SYMBOL",
    keytype   = "ENTREZID",
    multiVals = "first"
  )

  # mark genic
  mcols(gr)$genic[queryHits(hits)] <- TRUE

  # assign gene info
  mcols(gr)$gene_id[queryHits(hits)]     <- gene_ids
  mcols(gr)$gene_symbol[queryHits(hits)] <- symbols

  gr
}

In [ ]:
vcfs_indel_annotated <- endoapply(
  vcfs_indel,
  annotate_indels_coding,
  cds_gr = cds_gr
)

In [ ]:
vcfs_indel_annotated[[3]]

In [ ]:
vcfs_indel_genic <- endoapply(
  vcfs_indel,
  annotate_indels_genic,
  genes = genes
)

In [ ]:
vcfs_indel_genic[[3]]

In [ ]:
indel_coding_counts <- data.frame(
  sample = names(vcfs_indel_annotated),
  n_coding = sapply(
    vcfs_indel_annotated,
    function(gr) sum(gr$coding, na.rm = TRUE)
  ),
  row.names = NULL
)

In [ ]:
#table(unlist(vcfs_indel_annotated)$coding)
#sapply(vcfs_indel_annotated, function(gr) table(gr$coding))
table(unlist(vcfs_indel_genic)$genic)
sapply(vcfs_indel_genic, function(gr) table(gr$genic))


In [ ]:
vcfs_indel_coding_only <- endoapply(
  vcfs_indel_annotated,
  function(gr) gr[gr$coding]
)

In [ ]:
vcfs_indel_genic_only <- endoapply(
  vcfs_indel_genic,
  function(gr) gr[gr$genic]
)

In [ ]:
# get mutation loads
indel_loads = lengths(vcfs_indel_coding_only) %>% as.data.frame()
colnames(indel_loads) = "total_indels"
id_contexts = MutationalPatterns::get_indel_context(vcfs_indel_coding_only, ref_genome)
id_pks_contexts = lapply(id_contexts, select_context_indel, type = "Strelka")
indel_counts = count_indel_contexts(id_contexts)
in_pks_motif = lengths(id_pks_contexts)
indel_loads$in_pks_motif <- in_pks_motif[rownames(indel_loads)]
indel_loads$in_pks_motif[is.na(indel_loads$in_pks_motif)] <- 0L
indel_loads$fraction_pksmotif = indel_loads$in_pks_motif/indel_loads$total_indels
write.csv(indel_loads, file = paste0("colibactin_id_burden_coding.csv"), row.names = TRUE)



In [ ]:
# get mutation loads
indel_loads = lengths(vcfs_indel_genic_only) %>% as.data.frame()
colnames(indel_loads) = "total_indels"
id_contexts = MutationalPatterns::get_indel_context(vcfs_indel_genic_only, ref_genome)
id_pks_contexts = lapply(id_contexts, select_context_indel, type = "Strelka")
indel_counts = count_indel_contexts(id_contexts)
in_pks_motif = lengths(id_pks_contexts)
indel_loads$in_pks_motif <- in_pks_motif[rownames(indel_loads)]
indel_loads$in_pks_motif[is.na(indel_loads$in_pks_motif)] <- 0L
indel_loads$fraction_pksmotif = indel_loads$in_pks_motif/indel_loads$total_indels
write.csv(indel_loads, file = paste0("colibactin_id_burden_genic.csv"), row.names = TRUE)



Let's refit for signatures...

In [ ]:
# Need to get mut_mat of just coding SNVs

In [ ]:
vcfs_sbs_genic <- endoapply(
  vcfs_sbs,
  annotate_indels_genic,
  genes = genes
)

In [ ]:
vcfs_sbs_genic_only <- endoapply(
  vcfs_sbs_genic,
  function(gr) gr[gr$genic]
)

In [ ]:
vcfs_sbs_coding <- endoapply(
  vcfs_sbs,
  annotate_indels_coding,
  cds_gr = cds_gr
)

In [ ]:
vcfs_sbs_coding_only <- endoapply(
  vcfs_sbs_coding,
  function(gr) gr[gr$coding]
)

In [ ]:
mut_mat = mut_matrix(vcfs_sbs_coding_only, ref_genome)
fit_res = fit_to_signatures(mut_mat, as.matrix(sigs_known))
fit_res_clones_sbs = fit_res$contribution %>%
    prop.table(2) %>%
    as.data.frame() %>% rownames_to_column("Signature") %>%
    pivot_longer(cols = -Signature) %>%
    filter(Signature == "SBS88")
  fit_res_clones_sbs = merge(fit_res_clones_sbs, categories)

In [ ]:
write.csv(fit_res_clones_sbs, "SBS88_refit_coding.csv", row.names = FALSE)

In [ ]:
# get mutation loads
indel_loads = lengths(vcfs_indel_coding_only) %>% as.data.frame()
colnames(indel_loads) = "total_indels"
id_contexts = MutationalPatterns::get_indel_context(vcfs_indel_coding_only, ref_genome)
id_pks_contexts = lapply(id_contexts, select_context_indel, type = "Strelka")
indel_counts_coding = count_indel_contexts(id_contexts)

In [ ]:
id_sigs_select = id_signatures[, c("ID1", "ID2", "ID18")]
  fit_res_id = fit_to_signatures(indel_counts_coding, as.matrix(id_sigs_select))
  fit_res_clones = fit_res_id$contribution %>%
    prop.table(2) %>%
    as.data.frame() %>% rownames_to_column("Signature") %>%
    pivot_longer(cols = -Signature) %>%
    filter(Signature == "ID18")
  fit_res_clones = merge(fit_res_clones, categories)

In [ ]:
write.csv(fit_res_clones, "ID18_refit_coding.csv", row.names = FALSE)

In [ ]:
indel_loads = lengths(vcfs_indel_genic_only) %>% as.data.frame()
colnames(indel_loads) = "total_indels"
id_contexts = MutationalPatterns::get_indel_context(vcfs_indel_genic_only, ref_genome)
id_pks_contexts = lapply(id_contexts, select_context_indel, type = "Strelka")
indel_counts_genic = count_indel_contexts(id_contexts)

In [ ]:
id_sigs_select = id_signatures[, c("ID1", "ID2", "ID18")]
  fit_res_id = fit_to_signatures(indel_counts_genic, as.matrix(id_sigs_select))
  fit_res_clones = fit_res_id$contribution %>%
    prop.table(2) %>%
    as.data.frame() %>% rownames_to_column("Signature") %>%
    pivot_longer(cols = -Signature) %>%
    filter(Signature == "ID18")
  fit_res_clones = merge(fit_res_clones, categories)

In [ ]:
write.csv(fit_res_clones, "ID18_refit_genic.csv", row.names = FALSE)

Are there any driver mutations in our samples (snv or indel)?

In [ ]:
driverGenes = c("APC", "TP53", "KRAS", "BRAF", "PIK3CA", "SMAD4", "FBXW7", "TCF7L2","FAT4", "ATM")

In [ ]:
driver_gene_check <- data.frame(
  sample = names(contexts_coding),
  n_with_gene = sapply(
    contexts_coding,
    function(df) sum(df$gene_symbol_coding %in% driverGenes, na.rm = TRUE)
  ),
  driver_genes = sapply(
    contexts_coding,
    function(df) {
      hits <- unique(df$gene_symbol_coding[df$gene_symbol_coding %in% driverGenes])
      if (length(hits) == 0) NA_character_ else paste(hits, collapse = ";")
    }
  ),
  row.names = NULL
)

In [ ]:
driver_gene_check

In [ ]:
write.csv(driver_gene_check, "colibactin_driver_coding_gene_snvs.csv", row.names = FALSE)

In [ ]:
sum(driver_gene_check$n_with_gene > 0)

In [ ]:
driver_indel_summary <- data.frame(
sample = names(vcfs_indel_genic_only),
n_with_gene = sapply(
vcfs_indel_genic_only,
function(gr) sum(gr$gene_symbol %in% driverGenes, na.rm = TRUE)
),
driver_genes = sapply(
vcfs_indel_genic_only,
function(gr) {
hits <- unique(gr$gene_symbol[gr$gene_symbol %in% driverGenes])
if (length(hits) == 0) NA_character_ else paste(hits, collapse = ";")
}
),
row.names = NULL
)

In [ ]:
driver_indel_summary

In [ ]:
write.csv(driver_indel_summary, "colibactin_driver_gene_IDs.csv", row.names = FALSE)

In [ ]:
driver_indel_summary <- data.frame(
sample = names(vcfs_indel_coding_only),
n_with_gene = sapply(
vcfs_indel_coding_only,
function(gr) sum(gr$coding %in% driverGenes, na.rm = TRUE)
),
driver_genes = sapply(
vcfs_indel_coding_only,
function(gr) {
hits <- unique(gr$coding[gr$coding %in% driverGenes])
if (length(hits) == 0) NA_character_ else paste(hits, collapse = ";")
}
),
row.names = NULL
)

In [33]:
ID18_refit_coding <- read.csv("ID18_refit_coding.csv")
ID18_refit_genic <- read.csv("ID18_refit_genic.csv")
SBS88_refit_coding <- read.csv("SBS88_refit_coding.csv")
SBS88_refit_genic <- read.csv("SBS88_refit_genic.csv")
colibactin_driver_coding_gene_snvs <- read.csv("colibactin_driver_coding_gene_snvs.csv")
colibactin_driver_gene_IDs <- read.csv("colibactin_driver_gene_IDs.csv")
colibactin_driver_gene_snvs <- read.csv("colibactin_driver_gene_snvs.csv")
colibactin_id_burden_coding <- read.csv("colibactin_id_burden_coding.csv")
colibactin_id_burden_genic <- read.csv("colibactin_id_burden_genic.csv")
colibactin_snv_burden_coding <- read.csv("colibactin_snv_burden_coding.csv")
colibactin_snv_burden_genic <- read.csv("colibactin_snv_burden_genic.csv")



In [ ]:
colibactin_driver_coding_gene_snvs
colibactin_driver_gene_IDs
colibactin_driver_gene_snvs
colibactin_id_burden_coding
colibactin_id_burden_genic
colibactin_snv_burden_coding
colibactin_snv_burden_genic
ID18_refit_coding
ID18_refit_genic
SBS88_refit_coding
SBS88_refit_genic

In [34]:
df_list <- list(colibactin_driver_coding_gene_snvs, colibactin_driver_gene_IDs, colibactin_driver_gene_snvs, colibactin_id_burden_coding, colibactin_id_burden_genic,
colibactin_snv_burden_genic, ID18_refit_coding, ID18_refit_genic, SBS88_refit_coding, SBS88_refit_genic)

In [27]:
has_sample <- sapply(df_list, function(df) "sample" %in% names(df))

has_sample

[1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE

In [23]:
if (!all(has_sample)) {
  stop("These data frames are missing the 'sample' column: ",
       paste(names(df_list)[!has_sample], collapse = ", "))
}

ERROR: Error: These data frames are missing the 'sample' column: 


In [35]:
merged_df_base <- Reduce(function(x, y) merge(x, y, by = "sample", all = TRUE), df_list)

Warning message in merge.data.frame(x, y, by = "sample", all = TRUE):
“column names ‘Signature.x’, ‘injection.x’, ‘method.x’, ‘Signature.y’, ‘injection.y’, ‘method.y’ are duplicated in the result”


In [36]:
merged_df <- merge(merged_df_base, colibactin_snv_burden_coding, by = "sample", all = TRUE)

# Replace NAs (from the non-matching rows) with 0
merged_df[is.na(merged_df)] <- 0

Warning message in merge.data.frame(merged_df_base, colibactin_snv_burden_coding, :
“column names ‘Signature.x’, ‘injection.x’, ‘method.x’, ‘Signature.y’, ‘injection.y’, ‘method.y’ are duplicated in the result”


In [37]:
merged_df
write.csv(merged_df, "colibactin_coding_genic.csv", row.names = FALSE)

sample,n_driver_coding_snv,driver_coding_snv,n_with_driver_gene_id,driver_gene_id,n_with_driver_gene_snv,driver_genes_snv,total_coding_indels,in_pks_motif_coding,fraction_pksmotif_coding,⋯,Signature.x,SBS88_refit_coding,injection.x,method.x,Signature.y,SBS88_refit_genic,injection.y,method.y,n_AA_coding,n_with_coding_snv
<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<int>,<dbl>,⋯,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>
029_DC_T,0,0,0,0,0,0,1,0,0,⋯,SBS88,0.000000000,adenoma,Crypt,SBS88,0.000000000,adenoma,Crypt,0,0
030_CE_T,0,0,0,0,0,0,3,0,0,⋯,SBS88,0.000000000,adenoma,Crypt,SBS88,0.000000000,adenoma,Crypt,0,0
031_RE_T,0,0,0,0,0,0,2,0,0,⋯,SBS88,0.000000000,adenoma,Crypt,SBS88,0.036252773,adenoma,Crypt,0,0
058_RE_T,0,0,0,0,0,0,0,0,0,⋯,SBS88,0.000000000,normal,Crypt,SBS88,0.000000000,normal,Crypt,0,0
103_RE,0,0,1,FAT4,0,0,1,0,0,⋯,SBS88,0.000000000,normal,Crypt,SBS88,0.070442265,normal,Crypt,0,0
116_RE_T,0,0,0,0,0,0,0,0,0,⋯,SBS88,0.218341282,adenoma,Crypt,SBS88,0.156954405,adenoma,Crypt,1,19
116_TR,0,0,0,0,0,0,0,0,0,⋯,SBS88,0.000000000,adenoma,Crypt,SBS88,0.000000000,adenoma,Crypt,0,0
12b_D12_84_TR,0,0,0,0,0,0,0,0,0,⋯,SBS88,0.000000000,adenoma,Crypt,SBS88,0.000000000,adenoma,Crypt,0,0
148_SI,0,0,0,0,0,0,0,0,0,⋯,SBS88,0.000000000,normal,Crypt,SBS88,0.146749568,normal,Crypt,0,0
